In [30]:
import numpy as np
import pandas as pd
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_table('AUCs.txt')
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. 
Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [39]:
classifiers = ['C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf']
ans = [0, '', '']
for i in classifiers:
    for j in classifiers:
        if i != j and abs(stats.wilcoxon(data[i].values, data[j].values)[1]) > ans[0]:
            ans[0] = stats.wilcoxon(data[i].values, data[j].values)[0]
            ans[1] = i
            ans[2] = j
print(ans)

[6.5, 'C4.5', 'C4.5+m']


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [31]:
k = 0
for i in classifiers:
    for j in classifiers:
        if i != j and stats.wilcoxon(data[i].values, data[j].values)[1] > 0.05:
            k += 1
print(k)

4


Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

In [32]:
table = pd.DataFrame(columns=['0', '1', 'Wilcoxon stat', 'p-value'])
k = 0
for i in range(len(classifiers)):
    for j in range(i + 1, len(classifiers)):
        table.loc[k] = [classifiers[i]] + [classifiers[j]] + [stats.wilcoxon(data[classifiers[i]].values, data[classifiers[j]].values)[0]] + [stats.wilcoxon(data[classifiers[i]].values, data[classifiers[j]].values)[1]]
        k += 1
table

,0,1,Wilcoxon stat,p-value
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.0,0.022909


Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [37]:
from statsmodels.stats.multitest import multipletests
np.sum(multipletests(table['p-value'], method='holm')[0])

0

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [38]:
np.sum(multipletests(table['p-value'], method='fdr_bh')[0])

3